<a href="https://colab.research.google.com/github/naidu199/BrainTumar_Survival_Time_Analysis/blob/master/Dropping_Columns_with_Missing_Values_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
df =pd.read_csv("/content/drive/MyDrive/BrainTumarData/BrainTumor.csv")


In [15]:
df.columns

Index(['Patient ID', 'Age', 'Gender', 'Tumor Type', 'Tumor Grade',
       'Tumor Location', 'Treatment', 'Treatment Outcome',
       'Time to Recurrence (months)', 'Recurrence Site',
       'Survival Time (months)'],
      dtype='object')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Patient ID                   2000 non-null   int64  
 1   Age                          2000 non-null   int64  
 2   Gender                       2000 non-null   object 
 3   Tumor Type                   2000 non-null   object 
 4   Tumor Grade                  2000 non-null   object 
 5   Tumor Location               2000 non-null   object 
 6   Treatment                    2000 non-null   object 
 7   Treatment Outcome            2000 non-null   object 
 8   Time to Recurrence (months)  1438 non-null   float64
 9   Recurrence Site              1438 non-null   object 
 10  Survival Time (months)       2000 non-null   int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 172.0+ KB


In [17]:
df.isnull().sum()

Patient ID                       0
Age                              0
Gender                           0
Tumor Type                       0
Tumor Grade                      0
Tumor Location                   0
Treatment                        0
Treatment Outcome                0
Time to Recurrence (months)    562
Recurrence Site                562
Survival Time (months)           0
dtype: int64

In [18]:
df=df.drop(columns=['Patient ID','Time to Recurrence (months)','Recurrence Site'])

In [19]:
df.head()

,Age,Gender,Tumor Type,Tumor Grade,Tumor Location,Treatment,Treatment Outcome,Survival Time (months)
0,45,Male,Glioblastoma,IV,Frontal lobe,Surgery,Partial response,18
1,55,Female,Meningioma,I,Parietal lobe,Surgery,Complete response,36
2,60,Male,Astrocytoma,III,Occipital lobe,Surgery + Chemotherapy,Progressive disease,22
3,50,Female,Glioblastoma,IV,Temporal lobe,Surgery + Radiation therapy,Complete response,12
4,65,Male,Astrocytoma,II,Frontal lobe,Surgery + Radiation therapy,Partial response,48


In [20]:
df.dtypes

Age                        int64
Gender                    object
Tumor Type                object
Tumor Grade               object
Tumor Location            object
Treatment                 object
Treatment Outcome         object
Survival Time (months)     int64
dtype: object

In [21]:
unique_counts = df.nunique()
unique_counts

Age                       31
Gender                     2
Tumor Type                 3
Tumor Grade                4
Tumor Location             4
Treatment                  7
Treatment Outcome          4
Survival Time (months)    19
dtype: int64

In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# List of categorical columns
categorical_columns = ['Gender', 'Tumor Type', 'Tumor Grade', 'Tumor Location', 'Treatment', 'Treatment Outcome']

# Apply Label Encoding
for column in categorical_columns:
    df[column] = le.fit_transform(df[column])

df.head()

,Age,Gender,Tumor Type,Tumor Grade,Tumor Location,Treatment,Treatment Outcome,Survival Time (months)
0,45,1,1,3,0,3,1,18
1,55,0,2,0,2,3,0,36
2,60,1,0,2,1,4,2,22
3,50,0,1,3,3,6,0,12
4,65,1,0,1,0,6,1,48


In [28]:
#spliting featues and the target varibles
X=df.drop(columns=['Survival Time (months)'])
Y=df['Survival Time (months)']


In [29]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

In [30]:


def cross_validate_model(model, X, y, n_splits=5, random_state=42):
    """
    Perform K-Fold cross-validation on the given model.

    Parameters:
    - model: The regression model to be evaluated.
    - X: Feature DataFrame.
    - y: Target Series.
    - n_splits: Number of folds for cross-validation.
    - random_state: Random seed for reproducibility.

    Returns:
    - A tuple containing average Mean Squared Error and R² score.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    mse_scores = []
    r2_scores = []

    for train_index, test_index in tqdm(kf.split(X), total=kf.get_n_splits(), desc="Cross-Validation Progress"):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        mse_score = mean_squared_error(y_test, predictions)
        r2_score_value = r2_score(y_test, predictions)

        mse_scores.append(mse_score)
        r2_scores.append(r2_score_value)

        print(f"Fold Mean Squared Error: {mse_score:.4f}")
        print(f"Fold R² Score: {r2_score_value:.4f}")

    avg_mse = np.mean(mse_scores)
    avg_r2 = np.mean(r2_scores)

    print(f"Average Mean Squared Error: {avg_mse:.4f}")
    print(f"Average R² Score: {avg_r2:.4f}")

    return avg_mse, avg_r2


In [31]:
from sklearn.linear_model import LinearRegression


# Assuming X_scaled_df and y are already defined

# Test Linear Regression
print("Testing Linear Regression")
linear_model = LinearRegression()
result= cross_validate_model(linear_model, X, Y)

print("Linear Regression",  result)

Testing Linear Regression


Cross-Validation Progress: 100%|██████████| 5/5 [00:00<00:00, 59.82it/s]

Fold Mean Squared Error: 60.4515
Fold R² Score: 0.2076
Fold Mean Squared Error: 58.0156
Fold R² Score: 0.2077
Fold Mean Squared Error: 59.0901
Fold R² Score: 0.2637
Fold Mean Squared Error: 56.6597
Fold R² Score: 0.2366
Fold Mean Squared Error: 48.2605
Fold R² Score: 0.2609
Average Mean Squared Error: 56.4955
Average R² Score: 0.2353
Linear Regression (56.495465816769844, 0.23531063055740936)


In [32]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [33]:
# Define models
svm_model = SVR()
random_forest_model = RandomForestRegressor()
decision_tree_model = DecisionTreeRegressor()

# Test SVM Regression
print("Testing Support Vector Regression")
svm_result = cross_validate_model(svm_model, X, Y)
print("Support Vector Regression:", svm_result)

# Test Random Forest Regression
print("Testing Random Forest Regression")
rf_result = cross_validate_model(random_forest_model, X, Y)
print("Random Forest Regression:", rf_result)

# Test Decision Tree Regression
print("Testing Decision Tree Regression")
dt_result = cross_validate_model(decision_tree_model, X, Y)
print("Decision Tree Regression:", dt_result)


Testing Support Vector Regression


Cross-Validation Progress:  40%|████      | 2/5 [00:00<00:00,  3.78it/s]

Fold Mean Squared Error: 72.5747
Fold R² Score: 0.0487
Fold Mean Squared Error: 68.2450
Fold R² Score: 0.0680


Cross-Validation Progress:  80%|████████  | 4/5 [00:00<00:00,  4.76it/s]

Fold Mean Squared Error: 74.2504
Fold R² Score: 0.0748
Fold Mean Squared Error: 69.6623
Fold R² Score: 0.0614


...

Fold Mean Squared Error: 25.9099
Fold R² Score: 0.6509


Cross-Validation Progress: 100%|██████████| 5/5 [00:01<00:00,  3.32it/s]


Fold Mean Squared Error: 18.3257
Fold R² Score: 0.7193
Average Mean Squared Error: 19.8163
Average R² Score: 0.7316
Random Forest Regression: (19.816272343524123, 0.7315884506712266)
Testing Decision Tree Regression


Cross-Validation Progress: 100%|██████████| 5/5 [00:00<00:00, 114.11it/s]

Fold Mean Squared Error: 28.9476
Fold R² Score: 0.6206
Fold Mean Squared Error: 16.5654
Fold R² Score: 0.7738
Fold Mean Squared Error: 28.0750
Fold R² Score: 0.6502
Fold Mean Squared Error: 31.4353
Fold R² Score: 0.5765
Fold Mean Squared Error: 22.1734
Fold R² Score: 0.6604
Average Mean Squared Error: 25.4393
Average R² Score: 0.6563
Decision Tree Regression: (25.43932676666391, 0.656278359449397)


In [34]:
from xgboost import XGBRegressor

# Define XGBoost model
xgb_model = XGBRegressor()

# Test XGBoost Regression
print("Testing XGBoost Regression")
xgb_result = cross_validate_model(xgb_model, X, Y)
print("XGBoost Regression:", xgb_result)


Testing XGBoost Regression


Cross-Validation Progress:  20%|██        | 1/5 [00:00<00:01,  3.43it/s]

Fold Mean Squared Error: 21.5589
Fold R² Score: 0.7174


Cross-Validation Progress:  60%|██████    | 3/5 [00:00<00:00,  3.62it/s]

Fold Mean Squared Error: 13.2789
Fold R² Score: 0.8187
Fold Mean Squared Error: 20.1784
Fold R² Score: 0.7486


Cross-Validation Progress: 100%|██████████| 5/5 [00:01<00:00,  4.80it/s]

Fold Mean Squared Error: 23.8994
Fold R² Score: 0.6780
Fold Mean Squared Error: 18.2951
Fold R² Score: 0.7198
Average Mean Squared Error: 19.4421
Average R² Score: 0.7365
XGBoost Regression: (19.442144114878275, 0.7364893907267603)


In [38]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from tqdm import tqdm

def cross_validate_with_grid_search(model, param_grid, X, y, n_splits=5, random_state=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=kf,
        n_jobs=-1,
        verbose=1
    )

    grid_search.fit(X, y)

    print(f"Best parameters found: {grid_search.best_params_}")
    print(f"Best score found: {-grid_search.best_score_:.4f}")

    best_model = grid_search.best_estimator_

    # Evaluate the best model
    mse_scores = []
    r2_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        best_model.fit(X_train, y_train)
        predictions = best_model.predict(X_test)

        mse_score = mean_squared_error(y_test, predictions)
        r2_score_value = r2_score(y_test, predictions)

        mse_scores.append(mse_score)
        r2_scores.append(r2_score_value)

    avg_mse = np.mean(mse_scores)
    avg_r2 = np.mean(r2_scores)

    print(f"Average Mean Squared Error: {avg_mse:.4f}")
    print(f"Average R² Score: {avg_r2:.4f}")

    return best_model, avg_mse, avg_r2

# Define parameter grid for XGBoost
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}



# Define XGBoost model
xgb_model = XGBRegressor()

# Perform Grid Search
print("Testing XGBoost with Hyperparameter Tuning")
best_xgb_model, avg_mse, avg_r2 = cross_validate_with_grid_search(xgb_model, param_grid, X, Y)

print("XGBoost Regression:")
print(f"Average Mean Squared Error: {avg_mse:.4f}")
print(f"Average R² Score: {avg_r2:.4f}")


Testing XGBoost with Hyperparameter Tuning
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Best parameters found: {'colsample_bytree': 0.9, 'learning_rate': 0.2, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.9}
Best score found: 17.3124
Average Mean Squared Error: 17.3124
Average R² Score: 0.7661
XGBoost Regression:
Average Mean Squared Error: 17.3124
Average R² Score: 0.7661


In [44]:
X.shape

(2000, 7)

In [45]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd

# Define parameter grid for XGBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200], # Number of boosting rounds
    'max_depth': [3, 5, 7, 9], # Maximum depth of a tree
    'learning_rate': [0.01, 0.05, 0.1], # Step size shrinkage to prevent overfitting
    'subsample': [0.6, 0.8, 1.0], # Subsample ratio of the training instances
    'colsample_bytree': [0.6, 0.8, 1.0], # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2], # Minimum loss reduction required to make a further partition on a leaf node
    'reg_alpha': [0, 0.1, 0.5, 1], # L1 regularization term on weights
    'reg_lambda': [0, 0.1, 0.5, 1] # L2 regularization term on weights
}



# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
xgb_model = XGBRegressor()

# Initialize RandomizedSearchCV
randomized_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=50, # Number of parameter settings that are sampled
    scoring='r2', # Use R² score as evaluation metric
    cv=5, # Number of folds in cross-validation
    verbose=1,
    random_state=42
)

# Perform randomized search on the training data
randomized_search.fit(X_train, y_train)

# Print the best parameters and best R² score
print("Best parameters found: ", randomized_search.best_params_)
print("Best R² score: {:.2f}".format(randomized_search.best_score_))

# Get the best model
best_xgb_model = randomized_search.best_estimator_

# Make predictions on the test data using the best model
y_pred = best_xgb_model.predict(X_test)

# Calculate R² score and Mean Squared Error on test data
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Test R² score: {:.2f}".format(r2))
print("Test Mean Squared Error: {:.4f}".format(mse))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found:  {'subsample': 0.6, 'reg_lambda': 0.5, 'reg_alpha': 0.1, 'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.8}
Best R² score: 0.75
Test R² score: 0.75
Test Mean Squared Error: 19.1809
